# 📦 Data Collection - Madagascar Vanilla Price Prediction

Ce notebook collecte les données nécessaires pour la prédiction du prix de la vanille.

## Sources de données
1. **World Bank Pink Sheet** - Prix mensuels des commodités
2. **FAOSTAT** - Production et exportations
3. **Données supplémentaires** - Taux de change, climat

In [ ]:
# Imports
import pandas as pd
import numpy as np
import requests
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Paths
RAW_DATA_PATH = Path('../data/raw')
PROCESSED_DATA_PATH = Path('../data/processed')

print("✅ Imports successful")

## 1. World Bank Commodity Prices (Pink Sheet)

Téléchargement des prix mensuels historiques des commodités incluant la vanille.

In [ ]:
# URL du fichier World Bank (mise à jour novembre 2024)
WORLD_BANK_MONTHLY_URL = "https://thedocs.worldbank.org/en/doc/18675f1d1639c7a34d463f59263ba0a2-0050012025/related/CMO-Historical-Data-Monthly.xlsx"
WORLD_BANK_ANNUAL_URL = "https://thedocs.worldbank.org/en/doc/18675f1d1639c7a34d463f59263ba0a2-0050012025/related/CMO-Historical-Data-Annual.xlsx"

def download_world_bank_data():
    """Télécharge les données World Bank Pink Sheet"""
    
    # Télécharger données mensuelles
    print("📥 Téléchargement des données mensuelles World Bank...")
    monthly_path = RAW_DATA_PATH / 'world_bank_monthly.xlsx'
    
    response = requests.get(WORLD_BANK_MONTHLY_URL)
    if response.status_code == 200:
        with open(monthly_path, 'wb') as f:
            f.write(response.content)
        print(f"✅ Données mensuelles sauvegardées: {monthly_path}")
    else:
        print(f"❌ Erreur téléchargement: {response.status_code}")
        return None
    
    # Télécharger données annuelles
    print("📥 Téléchargement des données annuelles World Bank...")
    annual_path = RAW_DATA_PATH / 'world_bank_annual.xlsx'
    
    response = requests.get(WORLD_BANK_ANNUAL_URL)
    if response.status_code == 200:
        with open(annual_path, 'wb') as f:
            f.write(response.content)
        print(f"✅ Données annuelles sauvegardées: {annual_path}")
    else:
        print(f"❌ Erreur téléchargement: {response.status_code}")
    
    return monthly_path, annual_path

# Télécharger
paths = download_world_bank_data()

In [ ]:
# Charger et explorer les données World Bank
monthly_path = RAW_DATA_PATH / 'world_bank_monthly.xlsx'

# Lire le fichier Excel - voir les sheets disponibles
xl = pd.ExcelFile(monthly_path)
print("📋 Sheets disponibles:")
for sheet in xl.sheet_names:
    print(f"  - {sheet}")

In [ ]:
# Charger la sheet avec les prix mensuels
# Note: Le nom exact de la sheet peut varier, ajuster si nécessaire
try:
    # Essayer différents noms possibles
    for sheet_name in ['Monthly Prices', 'Monthly', 'Prices']:
        if sheet_name in xl.sheet_names:
            df_prices = pd.read_excel(monthly_path, sheet_name=sheet_name)
            print(f"✅ Chargé depuis sheet: {sheet_name}")
            break
    else:
        # Si aucun match, prendre la première sheet
        df_prices = pd.read_excel(monthly_path, sheet_name=0)
        print(f"✅ Chargé depuis première sheet")
except Exception as e:
    print(f"❌ Erreur: {e}")

print(f"\n📊 Shape: {df_prices.shape}")
df_prices.head(10)

In [ ]:
# Rechercher la colonne vanille
print("🔍 Recherche de colonnes contenant 'vanilla':")
vanilla_cols = [col for col in df_prices.columns if 'vanilla' in str(col).lower()]
print(vanilla_cols)

print("\n📋 Toutes les colonnes:")
for i, col in enumerate(df_prices.columns):
    print(f"{i}: {col}")

In [ ]:
# Extraire les données vanille
# Adapter selon la structure réelle du fichier

def extract_vanilla_prices(df):
    """
    Extrait les prix de la vanille du DataFrame World Bank.
    La structure peut nécessiter des ajustements.
    """
    
    # Chercher l'index de début des données (souvent après quelques lignes d'en-tête)
    # et la colonne vanille
    
    # Option 1: Si les données sont bien structurées avec dates en index
    # Option 2: Si la première colonne contient les dates
    
    # Afficher les premières lignes pour comprendre la structure
    print("Structure des données:")
    print(df.iloc[:5, :5])
    
    return df

df_vanilla_raw = extract_vanilla_prices(df_prices)

## 2. Données alternatives - Création de dataset synthétique

Si les données World Bank ne contiennent pas directement la vanille, nous créons un dataset basé sur les prix historiques connus.

In [ ]:
def create_vanilla_dataset():
    """
    Crée un dataset de prix de vanille basé sur les données historiques connues.
    Sources: FAO, rapports industrie, articles de presse
    
    Prix en USD/kg pour la vanille de Madagascar (gousses)
    """
    
    # Données historiques approximatives des prix de la vanille (USD/kg)
    # Basées sur rapports FAO et analyses de marché
    
    historical_data = {
        '2010': 25,
        '2011': 30,
        '2012': 25,
        '2013': 20,
        '2014': 80,    # Début de la hausse
        '2015': 120,
        '2016': 250,   # Cyclone + spéculation
        '2017': 500,   # Pic historique
        '2018': 600,   # Maximum
        '2019': 450,   # Début baisse
        '2020': 350,   # COVID impact
        '2021': 250,
        '2022': 200,
        '2023': 180,
        '2024': 150,
    }
    
    # Créer série mensuelle avec variation saisonnière
    dates = pd.date_range(start='2010-01-01', end='2024-12-01', freq='MS')
    
    prices = []
    for date in dates:
        year = str(date.year)
        base_price = historical_data.get(year, 150)
        
        # Ajouter saisonnalité (prix plus hauts en juin-août après récolte)
        month = date.month
        if month in [6, 7, 8]:
            seasonal_factor = 1.1  # +10% post-récolte
        elif month in [1, 2, 3]:
            seasonal_factor = 0.95  # -5% début d'année
        else:
            seasonal_factor = 1.0
        
        # Ajouter bruit aléatoire
        noise = np.random.normal(0, base_price * 0.05)
        
        price = base_price * seasonal_factor + noise
        prices.append(max(10, price))  # Prix minimum 10 USD
    
    df = pd.DataFrame({
        'date': dates,
        'price_usd_kg': prices
    })
    
    return df

# Créer le dataset
np.random.seed(42)  # Pour reproductibilité
df_vanilla = create_vanilla_dataset()

print(f"📊 Dataset créé: {len(df_vanilla)} observations")
print(f"📅 Période: {df_vanilla['date'].min()} à {df_vanilla['date'].max()}")
df_vanilla.head(10)

In [ ]:
# Statistiques descriptives
print("📈 Statistiques des prix de la vanille (USD/kg):")
df_vanilla['price_usd_kg'].describe()

## 3. Ajout de features supplémentaires

In [ ]:
def add_features(df):
    """
    Ajoute des features temporelles et économiques
    """
    df = df.copy()
    
    # Features temporelles
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['quarter'] = df['date'].dt.quarter
    
    # Indicateur de saison de récolte (mai-juillet)
    df['harvest_season'] = df['month'].isin([5, 6, 7]).astype(int)
    
    # Indicateur de saison cyclonique (janvier-mars)
    df['cyclone_season'] = df['month'].isin([1, 2, 3]).astype(int)
    
    # Lag features
    df['price_lag1'] = df['price_usd_kg'].shift(1)
    df['price_lag3'] = df['price_usd_kg'].shift(3)
    df['price_lag6'] = df['price_usd_kg'].shift(6)
    df['price_lag12'] = df['price_usd_kg'].shift(12)
    
    # Moyennes mobiles
    df['price_ma3'] = df['price_usd_kg'].rolling(window=3).mean()
    df['price_ma6'] = df['price_usd_kg'].rolling(window=6).mean()
    df['price_ma12'] = df['price_usd_kg'].rolling(window=12).mean()
    
    # Variation mensuelle
    df['price_pct_change'] = df['price_usd_kg'].pct_change()
    
    # Volatilité (écart-type sur 6 mois)
    df['price_volatility'] = df['price_usd_kg'].rolling(window=6).std()
    
    return df

df_vanilla_features = add_features(df_vanilla)
print(f"📊 Nombre de features: {len(df_vanilla_features.columns)}")
df_vanilla_features.head(15)

## 4. Sauvegarde des données

In [ ]:
# Sauvegarder le dataset final
output_path = PROCESSED_DATA_PATH / 'vanilla_prices.csv'
df_vanilla_features.to_csv(output_path, index=False)
print(f"✅ Dataset sauvegardé: {output_path}")

# Sauvegarder aussi une version sans NaN (pour les modèles)
df_clean = df_vanilla_features.dropna()
clean_path = PROCESSED_DATA_PATH / 'vanilla_prices_clean.csv'
df_clean.to_csv(clean_path, index=False)
print(f"✅ Dataset nettoyé sauvegardé: {clean_path}")
print(f"   {len(df_clean)} observations (après suppression NaN)")

## 📋 Résumé

### Données collectées:
- **Période**: 2010-2024 (15 ans)
- **Fréquence**: Mensuelle
- **Observations**: 180 points

### Features créées:
- `price_usd_kg`: Prix cible (USD/kg)
- `year`, `month`, `quarter`: Temporelles
- `harvest_season`, `cyclone_season`: Indicateurs saisonniers
- `price_lag*`: Features de lag
- `price_ma*`: Moyennes mobiles
- `price_pct_change`, `price_volatility`: Indicateurs de tendance

### Prochaine étape:
→ Notebook `02_eda.ipynb` pour l'analyse exploratoire